In [ ]:

# Importation all necessary libraries
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import json

In [ ]:
# Define the data directory
# Define the directories for training, validation, and testing datasets
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Define the transforms for the training, validation, and testing sets
# These transforms include resizing, cropping, normalizing, and converting to tensor
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])


#  Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_data = datasets.ImageFolder(valid_dir, transform=validation_transforms)
test_data = datasets.ImageFolder(test_dir ,transform = test_transforms)

# Define the dataloaders for training, validation, and testing datasets and load them in batches
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
vloader = torch.utils.data.DataLoader(validation_data, batch_size =32,shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 20, shuffle = True)


In [ ]:
# Load the category to name mapping
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load a pre-trained resnet50 model with default weights
model = models.resnet50(weights='DEFAULT')
# Freeze parameters so we don't backprop through them during training 
for param in model.parameters():
    param.requires_grad = False
    
# Define our new classifier to train
classifier = nn.Sequential(nn.Linear(2048, 512),
                           nn.ReLU(),
                           nn.Dropout(0.2),
                           nn.Linear(512, 102),
                           
                           nn.LogSoftmax(dim=1))
# Replace the model's classifier with our new classifier
model.fc = classifier
# Define the loss function 
criterion = nn.NLLLoss()
# Define the optimizer
# Only train the classifier parameters, feature parameters are frozen   
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# Move the model to the device (GPU or CPU)
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Define the number of epochs, steps, and running loss
epochs = 1
steps = 0
running_loss = 0
print_every = 5

# Training the model and loop over the epochs
for epoch in range(epochs):
    # Set the model to training mode
    model.train()
    # Loop over the training data
    for images, labels in trainloader:
        # Increment the steps
        steps += 1
        # Move the images and labels to the device
        images, labels = images.to(device), labels.to(device)
        
        # Zero the gradients of the optimizer
        optimizer.zero_grad()

        # Forward pass through the model
        logps = model(images)
        # Calculate the loss
        loss = criterion(logps, labels)
        # Backward pass to calculate gradients
        loss.backward()
        # Update the parameters using the optimizer
        optimizer.step()
        
        # Increment the running loss
        running_loss += loss.item()
        
        # Print the training loss every 'print_every' steps
        if  steps % print_every == 0:
            # Set the model to evaluation mode
            model.eval()
    
            test_loss = 0
            accuracy = 0

            # Evaluate the model on the validation set
            # Loop over the validation data
            for images, labels in vloader:
                # Move the images and labels to the device
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass through the model
                logps = model(images)
                # Calculate the loss
                batch_loss = criterion(logps, labels)
                # Increment the test loss
                test_loss += batch_loss.item()

                # Calculate the accuracy 

                ps = torch.exp(logps)
                # Get the prpobabilities and the top class
                top_p, top_class = ps.topk(1, dim=1)
                # Compare the top class with the labels
                equals = top_class == labels.view(*top_class.shape)
                # Calculate the accuracy
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
                # Print the training and validation loss and accuracy
                print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Validation loss: {test_loss/len(vloader):.3f}.. "
                  f"Validation accuracy: {accuracy/len(vloader):.3f}")
            
            # Reset the running loss
            running_loss = 0
            # Set the model back to training mode
            model.train()

Epoch 1/1.. Train loss: 4.656.. Validation loss: 4.480.. Validation accuracy: 0.113
Epoch 1/1.. Train loss: 4.395.. Validation loss: 4.281.. Validation accuracy: 0.152
Epoch 1/1.. Train loss: 4.414.. Validation loss: 4.067.. Validation accuracy: 0.215
Epoch 1/1.. Train loss: 4.132.. Validation loss: 3.803.. Validation accuracy: 0.258
Epoch 1/1.. Train loss: 3.812.. Validation loss: 3.570.. Validation accuracy: 0.287
Epoch 1/1.. Train loss: 3.904.. Validation loss: 3.305.. Validation accuracy: 0.328
Epoch 1/1.. Train loss: 3.234.. Validation loss: 3.068.. Validation accuracy: 0.326
Epoch 1/1.. Train loss: 3.167.. Validation loss: 2.813.. Validation accuracy: 0.376
Epoch 1/1.. Train loss: 3.081.. Validation loss: 2.525.. Validation accuracy: 0.445
Epoch 1/1.. Train loss: 2.979.. Validation loss: 2.344.. Validation accuracy: 0.482
Epoch 1/1.. Train loss: 2.616.. Validation loss: 2.128.. Validation accuracy: 0.546
Epoch 1/1.. Train loss: 2.633.. Validation loss: 1.967.. Validation accuracy